from Chapter 14 of _Hands on Machine Learning with Scikit-Learn and Tensorflow_

# Bare 2-step RNN 

In [1]:
import numpy as np
import tensorflow as tf

![](simpleRNN.png)

We're going to unroll it 2 steps and set the hidden vector equal to the output vector (a _very_ simple RNN). That is:

$$ Y_0 = \tanh(W_x X_0 + b) $$

and  

$$ Y_1 = \tanh(W_x X_1 + W_y Y_0 + b) $$

We'll pass batches of inputs in: a numpy array of size `[n_batches, n_inputs]`. That is, we're writing $X_t$ as a row vector, so really $Y_0 = \tanh(X_0^t W_x^t + b^t),$ but we'll just drop all the transposes. We should have:

* $X_0, X_1$: `[n_batches, n_inputs]`
* $W_x$: `[n_inputs, n_neurons]`
* $W_y$: `[n_neurons, n_neurons]`
* $b$: `[1, n_neurons] `

($b$ will be broadcast to `[n_batches, n_neurons]`). That means the output will be

* $Y_t$: `[n_batches, n_neurons]`

In [2]:
n_inputs = 3
n_neurons = 5

In [3]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

In [4]:
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

In [5]:
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

In [6]:
init = tf.global_variables_initializer()

In [7]:
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])

In [8]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

As a check, the following (first) output of the RNN should be a numpy array of size `[n_batches, n_neurons] = [4, 5]`.

In [10]:
print(Y0_val.shape)
print(Y0_val)

(4, 5)
[[ 0.8960636  -0.9961354   0.9960467  -0.9680858   0.44344255]
 [ 0.4309014  -1.          0.9999999  -0.9999996  -0.8754126 ]
 [-0.4852839  -1.          1.         -1.         -0.99660254]
 [-0.97851205 -0.999997   -0.969897   -0.34225827 -0.9942562 ]]


Same for the following (second) output.

In [11]:
print(Y1_val.shape)
print(Y1_val)

(4, 5)
[[-0.9999995  -1.          1.         -1.         -0.99999994]
 [-0.69430953  0.9957111  -0.9075036  -0.4842649  -0.95532596]
 [-0.99352264 -0.9999998   0.99997026 -1.         -0.99999714]
 [-0.9733558  -0.9990728   0.97920734 -0.9783926  -0.9841459 ]]


# Static 2-step Unrolling 

In [35]:
tf.reset_default_graph()

In [36]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

In [37]:
basic_cell = tf.keras.layers.SimpleRNNCell(units=n_neurons)  # replaces the deprecated tf.contrib.rnn.BasicRNNCell
output_seq, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32)

In [38]:
Y0, Y1 = output_seq

In [40]:
init = tf.global_variables_initializer()  # this needs to be defined here... defining it before the variables does bad things.
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [41]:
print(Y0_val.shape)
print(Y0_val)

(4, 5)
[[ 0.26076204 -0.87741923  0.9683349   0.87098503 -0.67625916]
 [ 0.9233953  -0.9907382   0.999999    0.9835779  -0.99912375]
 [ 0.99460393 -0.9993372   1.          0.998014   -0.99999803]
 [ 0.9606342   0.34861976  0.9994729  -0.9944234  -0.9970318 ]]


In [42]:
print(Y1_val.shape)
print(Y1_val)

(4, 5)
[[ 0.99945015 -0.9993708   1.          0.984867   -0.9999999 ]
 [ 0.13449268 -0.9358899   0.5924583   0.74528295 -0.60351944]
 [ 0.98775476 -0.9950965   0.99999994  0.941673   -0.9999885 ]
 [ 0.48508894 -0.589504    0.94871217  0.7335965  -0.9837607 ]]


# Static n-step Unrolling

Let's rewrite the above in a more scalable way. The input will be a placeholder of shape `[batch_size, n_steps, n_inputs]`, which in TensorFlow can be writte `[None, n_steps, n_inputs]`.

In [44]:
tf.reset_default_graph()

In [45]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [46]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

The second argument to `tf.contrib.rnn.static_rnn` should be the list `[X0, X1, ...]`. Each `X_j` is, as above, a tensor of shape `[batch_size, n_inputs]`, that is, a tensor of shape `[None, n_inputs]`. So we have to massage `X` into such a list. 

We can use `tf.unstack` for this; it turns a tensor of shape `(A, B, C)` into a list of tensors of shape `(B, C)`, where the `i`-th component of the list is the tensor at `A = i`. We want to unstack along `B` (so that we get shape `[None, n_inputs]`), so we'll also pass `axis=1`.

In [47]:
X_seqs = tf.unstack(X, axis=1)

In [48]:
basic_cell = tf.keras.layers.SimpleRNNCell(units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)

Let's check that each of the `n_steps` outputs has the expected shape `[None, n_neurons]`:

In [52]:
len(output_seqs)

2

In [53]:
output_seqs[0].get_shape()

TensorShape([Dimension(None), Dimension(5)])

Finally, we restack the outputs into a single tensor so that our output has shape `[None, n_steps, n_neurons]`.

In [49]:
outputs = tf.stack(output_seqs, axis=1)

In [50]:
outputs.get_shape()

TensorShape([Dimension(None), Dimension(2), Dimension(5)])

Let's try it out. Here's a batch of input data:

In [54]:
X_batch = np.array([
    [[0, 1, 2], [9, 8, 7]],  # instance 0 time steps 1 and 2
    [[3, 4, 5], [0, 0, 0]],  # instance 1
    [[6, 7, 8], [6, 5, 4]],  # instance 2
    [[9, 0, 1], [3, 2, 1]],  # instance 3
])

In [55]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [56]:
print(outputs_val)

[[[-0.9499117   0.14086019  0.6971834  -0.2441735   0.45512706]
  [-0.9373555  -0.99562085  0.9999227   0.99250203  0.99996305]]

 [[-0.9902277  -0.74542236  0.9922384   0.64982086  0.9879935 ]
  [-0.8743683   0.7260192  -0.69016814 -0.01985344 -0.6202365 ]]

 [[-0.9981246  -0.9684646   0.9998299   0.9467215   0.9998052 ]
  [-0.8056482  -0.86899126  0.9493427   0.9825388   0.9915183 ]]

 [[ 0.99998075 -0.9627134   0.98568964  0.03348969  0.99999964]
  [-0.05042231 -0.9036694   0.18797247  0.9902675   0.94957983]]]


# Dynamic Unrolling 

In [57]:
tf.reset_default_graph()

The following does the same thing as the above code (although `dynamic_rnn` does some magic during backprop and can swap gpu  to cpu memory to avoid OOM for large `n_steps` if you set `swap_memory=True`). 

It handles the stacking and unstacking internally, and takes an input of shape `[None, n_steps, n_inputs]` and outputs a shape `[None, n_steps, n_neurons]`.

In [63]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [59]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.keras.layers.SimpleRNNCell(units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

In [60]:
X_batch = np.array([
    [[0, 1, 2], [9, 8, 7]],  # instance 0 time steps 1 and 2
    [[3, 4, 5], [0, 0, 0]],  # instance 1
    [[6, 7, 8], [6, 5, 4]],  # instance 2
    [[9, 0, 1], [3, 2, 1]],  # instance 3
])

In [61]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

To verify: outputs_val should have shape `[4, 2, 5]`.

In [64]:
print(outputs_val)

[[[ 0.7926984   0.7797086  -0.6041814   0.9648911   0.27033582]
  [ 0.6572988  -0.56955177  0.9949784   1.         -0.63364404]]

 [[ 0.92245036  0.86741316 -0.02894654  0.99999976  0.31256512]
  [ 0.25936666 -0.5994076  -0.00476568  0.89737266 -0.15621978]]

 [[ 0.9722467   0.92173547  0.56612206  1.          0.35359406]
  [ 0.215779   -0.8811101   0.9424212   1.          0.15000457]]

 [[-0.5318223  -0.50594455  0.9999975   0.999985    0.93945324]
  [ 0.71124107 -0.6304598   0.83129096  0.99943846  0.72069645]]]


Boom.

# Variable-length Sequences 

We have to pass a list of sequence lengths to `dynamic_rnn` if we want to handle sequences of variable length. 

The input should still be of shape `[None, n_steps, n_inputs]`, but we set `n_inputs = max(seq_length)` and zero pad shorter sequences. So if we want to pass the four sequences
```
[0, 1, 2], [9, 8, 7], [6, 5, 4]  # seq of length 3
[1, 1, 0]  # seq of length 1
[1, 2, 3], [5, 8, 7]  # seq of length 2
[1, 2, 2], [3, 3, 1], [1, 2, 3]  # seq of length 3
```
we set our batch to be the following:

In [68]:
X_batch = np.array([
    [[0, 1, 2], [9, 8, 7], [6, 5, 4]],  # seq 0
    [[1, 1, 0], [0, 0, 0], [0, 0, 0]],  # seq 1
    [[1, 2, 3], [5, 8, 7], [0, 0, 0]],  # seq 2
    [[1, 2, 2], [3, 3, 1], [1, 2, 3]]
])
seq_length_batch = np.array([3, 1, 2, 3])

In [69]:
n_inputs = 3
n_steps = 3
n_neurons = 5

In [72]:
tf.reset_default_graph()

In [75]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
seq_length = tf.placeholder(tf.int32, [None])

basic_cell = tf.keras.layers.SimpleRNNCell(units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

In [77]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

In the outputs, we should see the second output (`Y_1`) a length-1 sequence of length-5 vectors with zero padding (i.e., 2 length-5 zero vectors). The third output (`Y_2`) should be a length-2 sequence with 1 zero vector.

In [79]:
print(outputs_val)

[[[-0.65343726  0.6189102   0.97039735 -0.1816915   0.6944628 ]
  [ 0.99998784  0.89341444  0.98185945 -1.          0.9999708 ]
  [ 0.9992925  -0.52434665  0.587328   -0.999998    0.98419636]]

 [[ 0.9017867  -0.4601119  -0.02660421 -0.8641541   0.38876387]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.09805908  0.7601289   0.99115115 -0.8503448   0.9087573 ]
  [ 0.99613225 -0.45343968  0.9999848  -0.99999875  0.999915  ]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.6046406   0.22202344  0.9688044  -0.90392137  0.8529464 ]
  [ 0.9960751  -0.97410154 -0.07563845 -0.9993654   0.8751569 ]
  [ 0.534737   -0.45697638  0.99398696 -0.97692496  0.9358953 ]]]


An important point: in this `SimpleRNNCell`, the hidden state is just the last output vector. So if we want the final states for each entry in the batch, we can just look at `states_val`:

In [80]:
print(states_val)

[[ 0.9992925  -0.52434665  0.587328   -0.999998    0.98419636]
 [ 0.9017867  -0.4601119  -0.02660421 -0.8641541   0.38876387]
 [ 0.99613225 -0.45343968  0.9999848  -0.99999875  0.999915  ]
 [ 0.534737   -0.45697638  0.99398696 -0.97692496  0.9358953 ]]


Notice that it handled the zero-padding correctly. It picked of the _final_ states given the `seq_length`s, not just the highest-index state.